![diagrama](https://github.com/Ironhack-Part-Time-Enero2022/apuntes-clases/blob/master/semana-19/imagenes/diagrama-sklearn.png?raw=true)


⚠️⚠️ [link](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) animado ⚠️⚠️

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
df = pd.read_csv("data/california_limpio_final.csv", index_col = 0)
df.head()

,median_home_val,median_household_income,ic2,ic3,ic4,ic5,avggift,x0_AL,x0_AR,x0_AZ,...,x0_NV,x0_OK,x0_OR,x0_SC,x0_TN,x0_TX,x0_WA,x0_WI,x1_F,x1_M
0,1158.437751,392.0,430.0,466.0,520.0,21975.0,10.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,537.000000,365.0,415.0,410.0,473.0,19387.0,5.666667,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,725.000000,301.0,340.0,361.0,436.0,18837.0,4.111111,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1158.437751,401.0,407.0,399.0,413.0,14014.0,10.000000,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,995.000000,252.0,280.0,316.0,348.0,17991.0,6.000000,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
df.describe().T.head()

,count,mean,std,min,25%,50%,75%,max
median_home_val,3333.0,838.382558,416.947970,128.0,544.0,770.0,1017.0,2135.0
median_household_income,3333.0,324.360636,114.365026,53.0,239.0,312.0,397.0,655.0
ic2,3333.0,371.184218,119.524798,72.0,282.0,360.0,449.0,720.0
ic3,3333.0,372.262826,115.458231,89.0,287.0,357.0,444.0,704.0
ic4,3333.0,415.028503,122.364189,94.0,324.0,402.0,489.0,768.0


In [5]:
y = df["median_home_val"]
X = df.drop(columns=["median_home_val"])

In [6]:
pipe = Pipeline(steps=[('estimator', LinearRegression())])

Debemos crearnos una lista, donde tendremos distintos diccionarios. En cada uno de ellos tendremos un modelo con sus correspondientes hiperparámetros. 

In [9]:
params_grid = [{
                'estimator':[LinearRegression()],
                },
                {
                'estimator': [DecisionTreeRegressor()],
                'estimator__max_depth': [4, 5, 6, 7],
                'estimator__max_features': ["sqrt", "log2"],
                'estimator__min_samples_split': [10, 50, 100], 
                'estimator__min_samples_leaf':[1,2,3],
                },
                {'estimator':[RandomForestRegressor()],
                 'estimator__n_estimators':[50,100,200,300],
                 'estimator__max_depth':[4, 5, 6, 7], 
                 'estimator__min_samples_split':[2,3], 
                 'estimator__min_samples_leaf':[1,2,3],
                },
                {'estimator':[KNeighborsRegressor()],
                 'estimator__n_neighbors':[3,5,6,7],
                }

              ]

Iniciamos el GridSearchCV que ya conociamos. 

In [10]:
grid = GridSearchCV(pipe, params_grid, n_jobs= -1, verbose = 3)

Entrenamos nuestro modelo

In [11]:
%%time
res = grid.fit(X, y)

Fitting 5 folds for each of 173 candidates, totalling 865 fits
CPU times: user 9.88 s, sys: 420 ms, total: 10.3 s
Wall time: 27min 3s


De todos los modelos entrenados, obtenemos el que mejor se ajuste a nuestros datos. 

In [16]:
best_model = grid.best_params_
best_model

{'estimator': RandomForestRegressor(max_depth=7),
 'estimator__max_depth': 7,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__n_estimators': 100}

Podemos sacar un dataframe con todos los modelos y sus métricas

In [76]:
results = pd.DataFrame(grid.cv_results_).head()
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator,param_estimator__max_depth,param_estimator__max_features,param_estimator__min_samples_leaf,param_estimator__min_samples_split,param_estimator__n_estimators,param_estimator__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018967,0.003041,0.005616,0.002201,LinearRegression(),NaN,NaN,NaN,NaN,NaN,NaN,{'estimator': LinearRegression()},0.524626,0.572560,0.575597,0.503767,0.556725,0.546655,0.028053,51
1,0.020229,0.002263,0.005314,0.000670,DecisionTreeRegressor(),1,sqrt,NaN,NaN,NaN,NaN,"{'estimator': DecisionTreeRegressor(), 'estima...",0.004814,-0.001257,0.325193,0.290522,0.235424,0.170939,0.141069,106
2,0.018497,0.001595,0.004534,0.000876,DecisionTreeRegressor(),1,log2,NaN,NaN,NaN,NaN,"{'estimator': DecisionTreeRegressor(), 'estima...",0.243285,0.311015,0.319691,0.123477,0.235424,0.246578,0.070408,105
3,0.025144,0.003713,0.003998,0.000671,DecisionTreeRegressor(),2,sqrt,NaN,NaN,NaN,NaN,"{'estimator': DecisionTreeRegressor(), 'estima...",0.287720,0.373397,0.317607,0.330910,0.335065,0.328939,0.027732,103
4,0.035112,0.003070,0.004539,0.000772,DecisionTreeRegressor(),2,log2,NaN,NaN,NaN,NaN,"{'estimator': DecisionTreeRegressor(), 'estima...",0.012438,0.018276,0.045186,0.434908,0.161476,0.134457,0.159598,107


A continuación aplicamos el metodo `cross_validate`

In [21]:
results = cross_validate(estimator=RandomForestRegressor(max_depth=7,
                        min_samples_split=2,min_samples_leaf = 2, n_estimators=100), 
               X=X, y=y, scoring='neg_root_mean_squared_error', cv=10, verbose = 3,  n_jobs= -1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:   17.9s remaining:   41.9s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   18.2s remaining:    7.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   26.3s finished


In [25]:
rmse_ = abs(results["test_score"].mean())
rmse_

280.7963803985328